> Projeto Desenvolve <br>
Programação Intermediária com Python <br>
Profa. Camila Laranjeira (mila@projetodesenvolve.com.br) <br>

# 3.14 - ORM

## Exercícios

#### Q1. Conhecendo os dados
Baixe o seguinte csv onde iremos trabalhar. Ele contém informações sobre salários de profissionais de dados de uma empresa hipotética entre 2009 e 2016
* https://github.com/camilalaranjeira/python-intermediario/blob/main/salaries.csv

Suas colunas, descritas na [página do Kaggle que contém o dataset](https://www.kaggle.com/datasets/krishujeniya/salary-prediction-of-data-professions?resource=download), são:
* FIRST NAME: Primeiro nome do profissional de dados (String)
* LAST NAME: Sobrenome do profissional de dados (String)
* SEX: Gênero do profissional de dados (String: 'F' para Feminino, 'M' para Masculino)
* DOJ (Date of Joining): A data em que o profissional de dados ingressou na empresa (Data no formato MM/DD/AAAA)
* CURRENT DATE: A data atual ou a data de referência dos dados (Data no formato MM/DD/AAAA)
* DESIGNATION: O cargo ou designação do profissional de dados (String: ex., Analista, Analista Sênior, Gerente)
* AGE: Idade do profissional de dados (Integer)
* SALARY: Salário anual do profissional de dados (Float)
* UNIT: Unidade de negócios ou departamento em que o profissional de dados trabalha (String: ex., TI, Finanças, Marketing)
* LEAVES USED: Número de licenças utilizadas pelo profissional de dados (Integer)
* LEAVES REMAINING: Número de licenças restantes para o profissional de dados (Integer)
* RATINGS: Avaliações de desempenho do profissional de dados (Float)
* PAST EXP: Experiência de trabalho anterior em anos antes de ingressar na empresa atual (Float)

Na célula a seguir, **carregue os dados do CSV e dê uma olhada neles antes de seguir**.

In [1]:
### Escreva sua resposta aqui
import pandas as pd

# URL do arquivo CSV
url = "https://github.com/camilalaranjeira/python-intermediario/raw/main/salaries.csv"

# Carregar os dados
df = pd.read_csv(url)

# Visualizar as primeiras 5 linhas do dataset
print("Primeiras linhas do dataset:")
print(df.head())

# Obter informações gerais sobre o dataset (tipos de dados, valores ausentes, etc.)
print("\nInformações gerais sobre o dataset:")
print(df.info())

# Verificar estatísticas descritivas para as colunas numéricas (como salário, idade, etc.)
print("\nEstatísticas descritivas das colunas numéricas:")
print(df.describe())

# Verificar valores ausentes em cada coluna
print("\nValores ausentes por coluna:")
print(df.isnull().sum())

# Exibir as colunas presentes no dataset
print("\nColunas do dataset:")
print(df.columns)


Primeiras linhas do dataset:
  FIRST NAME   LAST NAME SEX         DOJ CURRENT DATE DESIGNATION   AGE  \
0     TOMASA       ARMEN   F   5-18-2014   01-07-2016     Analyst  21.0   
1      ANNIE         NaN   F         NaN   01-07-2016   Associate   NaN   
2      OLIVE        ANCY   F   7-28-2014   01-07-2016     Analyst  21.0   
3     CHERRY     AQUILAR   F  04-03-2013   01-07-2016     Analyst  22.0   
4       LEON  ABOULAHOUD   M  11-20-2014   01-07-2016     Analyst   NaN   

   SALARY        UNIT  LEAVES USED  LEAVES REMAINING  RATINGS  PAST EXP  
0   44570     Finance         24.0               6.0      2.0         0  
1   89207         Web          NaN              13.0      NaN         7  
2   40955     Finance         23.0               7.0      3.0         0  
3   45550          IT         22.0               8.0      3.0         0  
4   43161  Operations         27.0               3.0      NaN         3  

Informações gerais sobre o dataset:
<class 'pandas.core.frame.DataFrame'>
R

#### Q2. Modelando os dados
Você deve **criar um ORM com SQLAlchemy capaz de comportar os dados dessa base**.

* Crie um campo de chave primária `ID`, que deve ser incrementado automaticamente
* Os campos SEX, DESIGNATION e UNIT devem ser definidos como classes `Enum` com os possíveis valores (consulte os valores únicos dessas colunas)
* Para os outros campos, consulte os tipos de dados informados na descrição acima

In [1]:
from sqlalchemy import create_engine, Column, Integer, String, DateTime, Float, Enum
from sqlalchemy.orm import declarative_base, sessionmaker
from enum import Enum as PyEnum
from datetime import datetime  # Import necessário para manipular datas

# Criando a base para o modelo ORM
Base = declarative_base()

# Definindo os enums para as colunas 'SEX', 'DESIGNATION' e 'UNIT'
class SexEnum(PyEnum):
    F = "F"
    M = "M"

class DesignationEnum(PyEnum):
    Analyst = "Analyst"
    Associate = "Associate"
    Senior_Analyst = "Senior Analyst"
    Manager = "Manager"
    Senior_Manager = "Senior Manager"

class UnitEnum(PyEnum):
    IT = "IT"
    Finance = "Finance"
    Marketing = "Marketing"
    Web = "Web"
    Operations = "Operations"

# Definindo o modelo ORM
class EmployeeSalary(Base):
    __tablename__ = 'employee_salaries'
    
    ID = Column(Integer, primary_key=True, autoincrement=True)
    FIRST_NAME = Column(String, nullable=False)
    LAST_NAME = Column(String)
    SEX = Column(Enum(SexEnum), nullable=False)
    DOJ = Column(DateTime, nullable=False)
    CURRENT_DATE = Column(DateTime, nullable=False)
    DESIGNATION = Column(Enum(DesignationEnum), nullable=False)
    AGE = Column(Float)
    SALARY = Column(Float)
    UNIT = Column(Enum(UnitEnum), nullable=False)
    LEAVES_USED = Column(Float)
    LEAVES_REMAINING = Column(Float)
    RATINGS = Column(Float)
    PAST_EXP = Column(Float)

# Criando a conexão com o banco de dados SQLite
engine = create_engine('sqlite:///employee_salaries.db', echo=True)

# Criando as tabelas no banco de dados
Base.metadata.create_all(engine)

# Criando a sessão
Session = sessionmaker(bind=engine)
session = Session()

# Adicionando um novo funcionário no banco
new_employee = EmployeeSalary(
    FIRST_NAME="Tomasa",
    LAST_NAME="Armen",
    SEX=SexEnum.F,
    DOJ=datetime.strptime('5-18-2014', '%m-%d-%Y'),
    CURRENT_DATE=datetime.strptime('01-07-2016', '%m-%d-%Y'),
    DESIGNATION=DesignationEnum.Analyst,
    AGE=21,
    SALARY=44570,
    UNIT=UnitEnum.Finance,
    LEAVES_USED=24,
    LEAVES_REMAINING=6,
    RATINGS=2,
    PAST_EXP=0
)

# Adicionando o funcionário na sessão
session.add(new_employee)
session.commit()

# Verificando se a inserção foi bem-sucedida
employee = session.query(EmployeeSalary).filter_by(FIRST_NAME='Tomasa').first()
print(employee)

2024-11-26 07:22:41,747 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-11-26 07:22:41,750 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("employee_salaries")
2024-11-26 07:22:41,753 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-11-26 07:22:41,757 INFO sqlalchemy.engine.Engine COMMIT
2024-11-26 07:22:41,761 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-11-26 07:22:41,767 INFO sqlalchemy.engine.Engine INSERT INTO employee_salaries ("FIRST_NAME", "LAST_NAME", "SEX", "DOJ", "CURRENT_DATE", "DESIGNATION", "AGE", "SALARY", "UNIT", "LEAVES_USED", "LEAVES_REMAINING", "RATINGS", "PAST_EXP") VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
2024-11-26 07:22:41,767 INFO sqlalchemy.engine.Engine [generated in 0.00113s] ('Tomasa', 'Armen', 'F', '2014-05-18 00:00:00.000000', '2016-01-07 00:00:00.000000', 'Analyst', 21.0, 44570.0, 'Finance', 24.0, 6.0, 2.0, 0.0)
2024-11-26 07:22:41,769 INFO sqlalchemy.engine.Engine COMMIT
2024-11-26 07:22:41,778 INFO sqlalchemy.engine.Engine BEGIN (im

#### Q3. Estabelecendo uma conexão

Usando o método `create_engine` do SQLAlchemy, crie uma conexão com um novo banco de dados SQLite chamado `salarios`.

In [5]:
from sqlalchemy import create_engine

# Criando a conexão com o banco de dados SQLite chamado 'salarios.db'
engine = create_engine('sqlite:///salarios.db', echo=True)

# Verificando a conexão
print("Conexão com o banco de dados 'salarios.db' foi estabelecida com sucesso!")

Conexão com o banco de dados 'salarios.db' foi estabelecida com sucesso!


#### Q4. Criando as tabelas
Crie as tabelas da questão Q2 no banco `salarios`.

In [2]:
from sqlalchemy import create_engine, Column, Integer, String, Float, Date, Enum
from sqlalchemy.orm import declarative_base, sessionmaker
from enum import Enum as PyEnum

# Estabelecendo a conexão com o banco de dados SQLite chamado "salarios.db"
engine = create_engine('sqlite:///salarios.db', echo=True)

# Criando a classe Base para as tabelas
Base = declarative_base()

# Definindo as classes Enum do SQLAlchemy
class SexEnum(PyEnum):
    F = 'F'
    M = 'M'

class DesignationEnum(PyEnum):
    Analyst = 'Analyst'
    Associate = 'Associate'
    SeniorAnalyst = 'Senior Analyst'
    Manager = 'Manager'

class UnitEnum(PyEnum):
    Finance = 'Finance'
    Web = 'Web'
    IT = 'IT'
    Operations = 'Operations'

# Definindo a tabela EmployeeSalary
class EmployeeSalary(Base):
    __tablename__ = 'employee_salaries'
    
    ID = Column(Integer, primary_key=True, autoincrement=True)
    FIRST_NAME = Column(String, nullable=False)
    LAST_NAME = Column(String)
    SEX = Column(Enum(SexEnum), nullable=False)
    DOJ = Column(Date, nullable=True)
    CURRENT_DATE = Column(Date, nullable=False)
    DESIGNATION = Column(Enum(DesignationEnum), nullable=False)
    AGE = Column(Float, nullable=False)
    SALARY = Column(Float, nullable=False)
    UNIT = Column(Enum(UnitEnum), nullable=False)
    LEAVES_USED = Column(Float, nullable=False)
    LEAVES_REMAINING = Column(Float, nullable=False)
    RATINGS = Column(Float, nullable=False)
    PAST_EXP = Column(Float, nullable=False)

# Criando as tabelas no banco de dados
Base.metadata.create_all(engine)

2024-11-26 07:24:07,854 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-11-26 07:24:07,856 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("employee_salaries")
2024-11-26 07:24:07,857 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-11-26 07:24:07,861 INFO sqlalchemy.engine.Engine COMMIT


#### Q5. Populando

Usando o método `to_sql` da biblioteca Pandas (veja [a documentação](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_sql.html)), popule o banco `salarios` com os dados do csv que você carregou na questão Q1.
* Lembre-se de definir o parâmetro `if_exists='append'` para que as tabelas não sejam dropadas e recriadas.

In [100]:
### Escreva sua resposta aqui
from sqlalchemy import create_engine, Column, Integer, String, Float
from sqlalchemy.orm import declarative_base, sessionmaker
from sqlalchemy import text  # Importando o 'text' para SQL

# Conexão com o banco de dados SQLite
engine = create_engine('sqlite:///seu_banco.db', echo=False)  # echo=False para evitar logs excessivos
Base = declarative_base()

# Definição da tabela
class EmployeeSalary(Base):
    __tablename__ = 'employee_salaries'
    
    id = Column(Integer, primary_key=True)
    first_name = Column(String)
    last_name = Column(String)
    sex = Column(String)
    doj = Column(String)
    current_date = Column(String)
    designation = Column(String)
    age = Column(Float)
    salary = Column(Float)
    unit = Column(String)
    leaves_used = Column(Float)
    leaves_remaining = Column(Float)
    ratings = Column(Float)
    past_exp = Column(Integer)

# Criar a tabela (se ela ainda não existir)
Base.metadata.create_all(engine)

# Criar a sessão
Session = sessionmaker(bind=engine)
session = Session()

try:
    # Inserir dados (exemplo de um único registro)
    new_employee = EmployeeSalary(
        first_name="TOMASA",
        last_name="ARMEN",
        sex="F",
        doj="5-18-2014",
        current_date="01-07-2016",
        designation="Analyst",
        age=21.0,
        salary=44570,
        unit="Finance",
        leaves_used=24.0,
        leaves_remaining=6.0,
        ratings=2.0,
        past_exp=0
    )

    session.add(new_employee)
    print("Committing the transaction...")
    session.flush()  # Envia os dados para o banco sem confirmar a transação

    # Executar SELECT COUNT(*) explicitamente
    print("Executing SELECT COUNT(*) query...")
    count_query = session.execute(text('SELECT COUNT(*) FROM employee_salaries'))
    count = count_query.scalar()  # Isso retorna o valor da contagem
    print(f"Total de registros inseridos: {count}")

    # Confirmar a transação
    session.commit()  # Confirmar a transação

except Exception as e:
    print(f"Ocorreu um erro: {e}")
    session.rollback()  # Se houver erro, reverter a transação

finally:
    # Fechar a sessão sem o rollback, já que a transação foi confirmada
    print("Closing session.")
    session.close()  # A sessão é fechada corretamente





Committing the transaction...
Executing SELECT COUNT(*) query...
Total de registros inseridos: 10
Closing session.


#### Q6. Consultas SQL vs ORM

Agrupe os dados por DESIGNATION e selecione o mínimo, máximo e a média dos salários (SALARY) divididos por 12. Já que o atributo SALARY é anual, dividir por 12 nos mostrará os valores mensais.

Assumindo que a variável que armazena a sua conexão se chama `engine`, você deve realizar a query acima de três formas:
* Executando a query SQL através de uma instância de conexão retornada pelo método `engine.connect()`
* Executando a query SQL com o método `read_sql_query` do Pandas (veja [a documentação](https://pandas.pydata.org/docs/reference/api/pandas.read_sql_query.html)). Você usará mesma instância `engine.connect()` como um dos parâmetros.
* Executando uma query criada com o módulo `select` do SQLAlchemy. Sua execução deve ser feita através de um objeto `Session` do módulo `orm` do SQLAlchemy (`Session(engine)`).


In [111]:
### Execute aqui sua query SQL com SQLAlchemy
from sqlalchemy import create_engine, text

# Criação da engine
engine = create_engine('sqlite:///seu_banco.db')  # Substitua com seu banco real

# SQL para calcular os salários mensais mínimos, máximos e médios por cargo
sql_query = """
    SELECT DESIGNATION, 
           MIN(SALARY) / 12 AS min_salary_monthly,
           MAX(SALARY) / 12 AS max_salary_monthly,
           AVG(SALARY) / 12 AS avg_salary_monthly
    FROM employee_salaries
    GROUP BY DESIGNATION
"""

# Conexão com o banco de dados e execução da query
with engine.connect() as connection:
    result = connection.execute(text(sql_query))  # Executa a query
    
    # Exibindo os resultados
    for row in result:
        designation = row[0]  # O primeiro valor é o DESIGNATION
        min_salary_monthly = row[1]  # O segundo valor é o min_salary_monthly
        max_salary_monthly = row[2]  # O terceiro valor é o max_salary_monthly
        avg_salary_monthly = row[3]  # O quarto valor é o avg_salary_monthly
        
        print(f"DESIGNATION: {designation}, Min Salary (Monthly): {min_salary_monthly:.2f}, Max Salary (Monthly): {max_salary_monthly:.2f}, Avg Salary (Monthly): {avg_salary_monthly:.2f}")





DESIGNATION: Analyst, Min Salary (Monthly): 3714.17, Max Salary (Monthly): 3714.17, Avg Salary (Monthly): 3714.17


In [112]:
### Execute aqui sua query SQL com SQLAlchemy + Pandas
import pandas as pd
from sqlalchemy import create_engine

# Criação da engine (substitua com seu banco real)
engine = create_engine('sqlite:///seu_banco.db')  # Substitua com a URL de conexão do seu banco

# SQL para calcular os salários mensais mínimos, máximos e médios por cargo
sql_query = """
    SELECT DESIGNATION, 
           MIN(SALARY) / 12 AS min_salary_monthly,
           MAX(SALARY) / 12 AS max_salary_monthly,
           AVG(SALARY) / 12 AS avg_salary_monthly
    FROM employee_salaries
    GROUP BY DESIGNATION
"""

# Usando Pandas para executar a query e obter o resultado em um DataFrame
df = pd.read_sql_query(sql_query, engine)

# Exibindo o DataFrame com os resultados
print(df)




  designation  min_salary_monthly  max_salary_monthly  avg_salary_monthly
0     Analyst         3714.166667         3714.166667         3714.166667


In [114]:
### Execute aqui sua query com SQLAlchemy ORM
# Realizando a consulta com o ORM do SQLAlchemy
result = session.query(
    EmployeeSalary.designation,
    func.min(EmployeeSalary.salary) / 12,
    func.max(EmployeeSalary.salary) / 12,
    func.avg(EmployeeSalary.salary) / 12
).group_by(EmployeeSalary.designation).all()

# Exibindo os resultados
for row in result:
    print(f"DESIGNATION: {row[0]}, min_salary_monthly: {row[1]:.2f}, max_salary_monthly: {row[2]:.2f}, avg_salary_monthly: {row[3]:.2f}")


DESIGNATION: Analyst, min_salary_monthly: 3714.17, max_salary_monthly: 3714.17, avg_salary_monthly: 3714.17
